# Linear Programming via Interior Point Method
Attempt number one, yay
Heavy referening to 
Anderson and Anderson

### References
[R1] High Performance Optimzation

[R2] Freund MIT OCW

### Symmetric Linear System Solver for the Newton Equation System
The Newton Equation system (R1, Eqn. 8.25) motivates the need to solve a system of linear equations of the form

$$\left[\begin{array}{cc}
    X^{-1}S & A^T \\
    A & 0 \\
\end{array} \right]
\left[ \begin{array}{cc}
    u \\ v \\
\end{array} \right]
=
\left[ \begin{array}{cc}
    r_1 \\ r_2 \\
\end{array}\right]$$
$r_1$ and $r_2$ are two arbitrary vectors *(R1, Eqn. 8.30)* and where we call

$$K:=\left[\begin{array}{cc}
    X^{-1}S & A^T \\
    A & 0 \\
\end{array} \right] $$

This linear system can be reduced to a system of normal equations (8.31) and (8.32)
$$Mv = AD^{-1}A^Tv = r_2 + AD^{-1}r_1$$
$$u = D^{-1}(A^Tv-r_1)$$
where $D:=X^{-1}S$. Note that $X$ is a diagonal matrix with $x_j$ as its elements, and $S$ is a diagonal matrix with $s_j$ as its elements.

We note that $M = AD^{-1}A^T$ is positive definition and therefore there exists a Cholesky decomposition $M=CC^*$. The linear equation (8.31) can then easily be solved after forward and backward substitution.

**LIBBY TODO**
  * Look into Wright [R1, Ref 39] for how (8.31) and (8.32) were derviced using the normal equation approach)
  * Forward and back substitution of Cholesky: See PS4, Assgn. 1.3 -> ``` subs!(C', x, true), subs!(C, x, false) ```
  * Cholesky: PS5: We implemented chol!, imcomplete chol, and the icc for special 2d poisson matrix

In [ ]:
function sym_solver(Dinv, M, A, r1, r2)
    # r <= r2 + A*(Dinv*r1)
    r = r2 + BLAS.gemv('N',1,A,Dinv*r1)
    
    v = M \ r
    u = Dinv*(BLAS.gemv('T',1,A,v) - r1)
    
    return u, v
end

In [ ]:
function calcStepSize(x,Δx,s,Δs,τ,Δτ,κ,Δκ,α0)
    # [R1] Section 4.3: The Step Size
    # Take the step size for both primal and dual space
    # Different step sizes as outlined by [R1: Ref41] and in OCW notes
    # α0 = 0.9999 normally, α0 = 1 for Mehrota predictor
    
    # Only calculate for negative Δs
    # TODO: Read it somewhere, but not mentioned in R1 Section 4.3.... Maybe in OCW notes?
    ix = (Δx.< 0)
    is = (Δs.< 0)
    
    αx = any(ix) ? α0 * minimum(-x[ix]./Δx[ix]) : 1
    αs = any(is) ? α0 * minimum(-s[is]./Δs[is]) : 1
    ατ = Δτ < 0  ? -α0 * τ / Δτ : 1
    ακ = Δκ < 0  ? -α0 * κ / Δκ : 1
    
    α = minimum([1, αx, αs, ατ, ακ])
    
    return α
end

In [103]:
# [R1] Section 5. The Solution of the Newton Equation System
function getSearchDirection(A,b,c,x,y,s,τ,κ,μ,γ,rp,rd,rg,pc)    
    # All input parameters as expected
    # If using pc: γ = 0, else: mean(s.*x) and decreases by β=0.1 each iteration

    μ = (vecdot(x,s) + τ*κ)/(size(x,1)+1)

    # Assemble M from eqn 8.31
    # D⁻¹ = X⁻¹S, but X and S are diagonal matrices
    Dinv = x.\s
    M = A*(Dinv*A')
    # Upgrades: If A is sparse, we can use special solver
    # Upgrades: If A is cholesky, we can decompose M later into CᵀC

    # If pc option on, perform Mehrotra's predictor-corrector strategy
    n_pc = pc ? 1 : 0
    i = 0

    α, Δx, Δs, Δτ, Δκ = 0, 0, 0, 0, 0
    while i <= n_pc
        # Setup residuals for linear system search direction
        # [R1] Eqn 8.6
        r̂p = (1-γ)*rp
        r̂d = (1-γ)*rd
        r̂g = (1-γ)*rg
        # [R1] Eqn 8.7
        r̂xs = γ*μ*ones(n,1)-x.*s
        r̂tk = γ*μ-τ*κ

        # Final search direction with Mehrotra predictor-correction
        # [R1] Eqn. 8.13
        if i == 1
            r̂xs .-= Δx.*Δs
            r̂tk .-= Δτ *Δκ
        end

        solved = false

        while !solved
            # [R1], Eqns 8.28 and 8.29
            try
                p, q = sym_solver(Dinv, M, A, c, b)
                u, v = sym_solver(Dinv, M, r̂d-x.\r̂xs, r̂p)

                # TODO must check if p and q are NOT nan    
                solved = true
            catch
                println("Warning: Unable to solve for (p,q) or (u,v). According to Python code, usually due to redundant constraints or when approaching solution. Change solver.")
                break
            end
        end

        # rg should be a scalar already, we hope
        # Now that we have solved for the matrix K, we can solve the Newton system: (8.26) and then (8.25)
        Δτ = (r̂g + r̂tk/τ + vecdot(c,u) - vecdot(b,v)) / (κ/τ - vecdot(c,p), + vecdot(b,q))
        Δx = u + p*Δτ
        Δy = v + q*Δτ
        Δs = x.\(r̂xs-s.*Δx)
        Δκ = (r̂tk - κ*Δτ)/τ

        # Calculate maximal step size so that we may choose γ, use α0 = 1 for Mehrota predictor
        # [R1] Eqn. 8.12
        α = calcStepSize(x,Δx,s,Δs,τ,Δτ,κ,Δκ,1)
        γ = (1-α)^2 * min(0.1, 1-α)

        i += 1
    end
    
    return Δx, Δy, Δs, Δτ, Δκ
end

getSearchDirection (generic function with 1 method)

In [120]:
function lp_ip(A,b,c,kmax, pc, α0=0.9999, β = 0.1, tol=1e-8)
    # Solves a linear pogramming problem in standard form:
    #   (P) min cᵀx s.t. Ax = b, x⩾0
    # This problem has the dual
    #   (D) max bᵀy s.t. Aᵀy+s = c, s⩾0
    
    # Parameters
    #  A     nxn array
    #  b     nx1 array, RHS of equality constraint
    #  c     nx1 array, coefficients of linear objective function to be minimized
    #  kmax  integer, maximum number of iterations
    #  pc    boolean, true if using Mehrotra's predictor-corrector strategy
    #  α₀    float, maximal step size as suggested in [R1] Table 8.3 and in [R2] <- (I THINK it's called 'r')
    #  β     float, reductation in path parameter each iteration, 0.1 as suggested by [R2]
    #  tol   float, termination tolerance
    
    m, n = size(A)
    k = 0;
    
    # Initialize starting point, proposed by (X. Xu, PF Hung, and Y. Ye., 1996), referenced in R1 and R2
    x   = ones(n,1)
    y,s = 0, ones(n,1)
    τ,κ  = 1, 1

    # Calculate baseline residuals (these are simplified given that we know the vectors)
    rp = b*τ-A*x
    rd = c - s
    rg = κ + vecdot(c,x)
    nrp₀ = max(1, norm(rp))
    nrd₀ = max(1, norm(rd))
    nrg₀ = max(1, norm(rg))
    
    relres = zeros(kmax+1,3)
    relres[1,:] = [norm(rp)/nrp₀,norm(rd)/nrd₀, norm(rg)/nrg₀]
    
    # TODO There is a third stopping criterion, figure out which one to use...
    # Freund: sᵀ*x < tol
    # Anderson: rA = abs(cᵀ*x - bᵀ*y)/(τ+abs(bᵀ*y))
    while k < kmax && relres[k+1,1] > tol && relres[k+2,2]>tol
        k .+= 1
        γ = pc ? 0 : β*mean(s.*x)
        
        # Solve Newton equation
        Δx, Δy, Δs, Δτ, Δκ = getSearchDirection(A,b,c,x,y,s,τ,κ,γ,rp,rd,rg)
        
        # Make step
        α = calcStepSize(x,Δx,s,Δs,τ,Δτ,κ,Δκ,α0)
        x .+= αp * Δx
        y .+= αd * Δy
        s .+= αd * Δs
        τ .+= αp * Δτ
        κ .+= αd * Δκ
        
        # Update residuals
        rp = b*τ-A*x
        rd = c*τ - BLAS.gemv('T',1,A,y) - s
        rg = κ + vecdot(c,x) - vecdot(b,y)
        
        relres[k+1,:] = [norm(rp)/nrp₀,norm(rd)/nrd₀, norm(rg)/nrg₀]
        
        # [R1] Check if primal or dual infeasible
        if μ <= tol*μ₀ && τ <= tol*min(1,κ)
            # Break
        end
        # THERE IS A SECOND INFEASIBILITY CRITERIA, GO CHECK IT OUT (p210)
    end
    
end

lp_ip (generic function with 2 methods)

In [ ]:
function lp_ip_simple(A,b,c, kmax, tol = 1e-8)
    # Solves a linear pogramming problem in standard form:
    #   (P) min cᵀx s.t. Ax = b, x⩾0
    # This problem has the dual
    #   (D) max bᵀy s.t. Aᵀy+s = c, s⩾0
    
    # Based on simpler algorithm presented by Freund 2004
    
    r = 0.99
    k = 0 
    n = size(b)
    
    # Initialize starting point
    # Proposed by (X. Xu, PF Hung, and Y. Ye., 1996), referenced in R1 and R2
    x   = ones(n,1)
    y,s = 0, ones(n,1)
    τ,κ  = 1, 1
    
    relres = zeros(kmax+1, 3)
    relres[1,:] = [norm(A*x - b), norm(A'*y+s-c), norm(vecdot(s,x))]
    
    while k < kmax && relres[k+1,1] > tol && relres[k+1,2] > tol && relres[k+1,3] > tol
        k += 1
        
        μ = 0.1* vecdot(x,s)/k
        
        rp = b - A*x
        rd = c - A'*y - s
        rg = μ*ones(n,1) - (x.*s) # DOUBLE CHECK THIS, Se = s, so elementwise multiplication
        
        # Solve Newton equation for Δs
        Ainv = inv(A)
        ytemp = (s./x)*Δx + rd + rg./x
        Δx = Ainv*rp
        Δy = -Ainv'* ytemp
        Δs = r2 - ytemp
        
        # Calculate step sizes
        αp = min(1,r*min(-x./Δx)) # inside does elementwise division
        αd = min(1,r*min(-s./Δs))
        
        # Update values
        x .+= αp*Δx
        y .+= αd*Δy
        s .+= αd*Δs
        
        # Calculate relative improvements
        relres[k+1,:] = [norm(A*x - b), norm(A'*y+s-c), norm vecdot(s,x)]
    end
end

In [4]:
# Simple test function, from MATLAB linprog documentation
# Optimal solution should be: x* = [0.6667; 1.3333]
A = [1 1; 1 1/4; 1 -1; -1/4 -1; -1 -1; -1 1];

b = [2 1 2 1 -1 2];

c = [-1 -1/3]

lp_ip_simple(A,b,c,)

3-element Array{Int64,1}:
 5
 8
 9